In [19]:
import re, nltk
import cleantext as ct
import pandas as pd
from itertools import chain
from nltk.corpus import stopwords
from nltk import regexp_tokenize

In [20]:
def clean(csv):
  df = pd.read_csv(csv)
  for i in range(len(df["content"])):
    df.loc[i, "content"] = ct.clean(df.loc[i, "content"], lower=True, no_line_breaks=True, no_numbers=True, no_emails=True, no_urls=True, replace_with_number="<NUM>")
  return df

clean("news_sample.csv").to_csv("cleaned_news_sample.csv")

In [21]:
def tokenize(stringInput):
              
    return regexp_tokenize(stringInput, pattern=r"\s|[.,;']", gaps=True)


In [22]:
def stem(input):
  stemmer = nltk.SnowballStemmer(language="english", ignore_stopwords=True)
  stem_words = []
  for token in input:
    x = stemmer.stem(token)
    stem_words.append(x)
  return stem_words

In [23]:
def remove_stopwords(cell):
  for token in cell:
    stop_words = stopwords.words('english')
    filteredInput = []
    
    for token in cell:
      if token not in stop_words:
        filteredInput.append(token)
        
    return filteredInput

In [24]:
def run():
  data = "news_sample.csv"
  cleaned = clean(data)
  tokenizedData = tokenize(cleaned)
  stopwordsRemoved = remove_stopwords(tokenizedData)
  StemmedData = stem(stopwordsRemoved)
  
  for x in StemmedData:
    print(x)

run()

TypeError: expected string or bytes-like object, got 'DataFrame'